In [1]:
import pandas as pd
import numpy as np

In [11]:
hcmst = pd.read_csv("hcmst2017.csv", encoding="mac_roman")
hcmst.head()

,CaseID,CASEID_NEW,qflag,weight1,weight1_freqwt,weight2,weight1a,weight1a_freqwt,weight_combo,weight_combo_freqwt,...,hcm2017q24_met_through_family,hcm2017q24_met_through_friend,hcm2017q24_met_through_as_nghbrs,hcm2017q24_met_as_through_cowork,w6_subject_race,interracial_5cat,partner_mother_yrsed,subject_mother_yrsed,partner_yrsed,subject_yrsed
0,2,2014039,Qualified,NaN,NaN,0.8945,NaN,NaN,0.277188,19240,...,no,no,no,no,White,no,12.0,14.0,12.0,14.0
1,3,2019003,Qualified,0.9078,71115.0,NaN,0.9026,70707.0,1.020621,70841,...,no,no,no,yes,White,no,12.0,16.0,17.0,17.0
2,5,2145527,Qualified,0.7205,56442.0,NaN,0.7164,56121.0,0.810074,56227,...,no,no,no,no,White,no,9.0,7.5,14.0,17.0
3,6,2648857,Qualified,1.2597,98682.0,1.3507,1.2524,98110.0,0.418556,29052,...,no,no,no,no,White,no,16.0,12.0,12.0,12.0
4,7,2623465,Qualified,0.8686,68044.0,NaN,0.8636,67652.0,0.976522,67781,...,no,no,yes,no,White,no,14.0,17.0,16.0,16.0


In [17]:
cols = ["ppage", "Q5", "Q21A_Year", "Q21A_Month", "Q21B_Year", "Q21B_Month", "Q21C_Year", "Q21C_Month", "Q21D_Year", "Q21D_Month"]
colnames = ["age", "same_sex", "year_met", "month_met", "year_dating", "month_dating", "year_movein", "month_movein", "year_marry", "month_marry"]
relevant = hcmst[cols]
relevant.columns = colnames
relevant = relevant.loc[relevant["year_met"].notna()]
relevant.head()


,age,same_sex,year_met,month_met,year_dating,month_dating,year_movein,month_movein,year_marry,month_marry
1,55,NaN,1983,May,1995,August,1996,February,1996,February
2,47,NaN,2006,January,2006,June,2006,July,2008,May
4,59,NaN,1983,September,1983,October,1984,August,1984,August
5,59,"No, we are an opposite-sex couple",1981,December,1982,June,1982,June,1982,June
6,66,NaN,1966,October,1967,January,1969,August,1969,August


In [15]:
def month_year(row):
    year = row[0]
    if pd.isna(year) or year == "Refused":
        return np.nan
    month = "" if row[1] == "Refused" or pd.isna(row[1]) else row[1] + " "
    return month + year


In [16]:
clean = pd.DataFrame()
clean["age"] = relevant["age"]
clean["same_sex"] = relevant["same_sex"].apply(
    lambda x: 0 if str(x).startswith("No") else -1 if pd.isna(x) else 1)
for stage in ["met", "dating", "movein", "marry"]:
    clean[stage] = relevant[["year_"+stage, "month_"+stage]].apply(month_year, axis=1)
clean.head()

,age,same_sex,met,dating,movein,marry
1,55,-1,May 1983,August 1995,February 1996,February 1996
2,47,-1,January 2006,June 2006,July 2006,May 2008
4,59,-1,September 1983,October 1983,August 1984,August 1984
5,59,0,December 1981,June 1982,June 1982,June 1982
6,66,-1,October 1966,January 1967,August 1969,August 1969


In [21]:
# clean = clean.loc[clean["same_sex"] != -1].sample(80)
clean = clean.sort_values("age")
clean

,age,same_sex,met,dating,movein,marry
655,18,-1,August 2015,January 2016,NaN,NaN
1289,18,-1,November 2014,January 2015,November 2016,NaN
3433,18,-1,April 2012,February 2016,NaN,NaN
3295,18,-1,January 2013,February 2013,NaN,NaN
874,18,0,August 2016,November 2016,May 2017,May 2017
1393,18,-1,November 2012,November 2016,NaN,NaN
2665,18,0,October 2000,October 2000,November 2000,October 2000
649,18,-1,April 2017,April 2017,NaN,NaN
1530,18,-1,June 2017,July 2017,NaN,NaN
1707,18,-1,November 2016,July 2017,NaN,NaN


In [22]:
clean.to_csv("../data/time-data.csv", na_rep="null", index=False)